In [71]:
import json
import string
import nltk
import pandas as pd
import numpy as np
import itertools

In [72]:
business = pd.read_json('business_city.json', lines=True)
review = pd.read_json('review_city.json', lines=True)
tip = pd.read_json('tip_city.json', lines=True)
user = pd.read_json('user_city.json', lines=True)

In [73]:
raw_business = pd.merge(review,business, how="left",on="business_id")
raw_business.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11,Deagan's Kitchen & Bar,...,OH,44107,41.485192,-81.800145,4.0,417,1,"{'BusinessAcceptsCreditCards': 'True', 'Outdoo...","Restaurants, Gastropubs, Nightlife, American (...","{'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22..."
1,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5,1,0,0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,Black Cab & Limo,...,IL,,40.116420,-88.243383,4.5,28,1,{'WiFi': 'u'free''},"Airport Shuttles, Transportation, Taxis, Hotel...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
2,Ne_2CSfcKIqXHmv_KC0uqg,6sJN_HlM_uwpfLJ1puf1Fg,oVuZtlCFg_zF090Nhtp0Ow,4,3,0,1,A close friend was in town and so instead of t...,2018-02-01 19:15:00,Choolaah Indian BBQ,...,PA,15206,40.460266,-79.923461,4.0,302,1,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Indian, Vegetarian, Vegan, Gluten...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-17:0', ..."
3,2mLlQ9n5Ln_fykOuX_b0-A,EZ6g9a17UdMjtR-wrZebmA,wSAGILK2CFbq5BVZ5D67-A,3,0,0,0,The service is attentive. The decor attractive...,2015-06-21 00:07:59,Adega,...,OH,44115,41.499615,-81.685922,4.0,201,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Mediterranean, Hotels, Beer, Wine & Spirits, B...","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
4,Ymb8ZYPg2e8dvqhdj3Gkqw,hHFSpKcnDtJOs6M-JKFZ8Q,oCvxJBOs-Mbg-4oanPO7lQ,4,0,0,0,The ingredients are always fresh and I like th...,2016-10-26 14:53:42,Pita Pit,...,PA,15203,40.426647,-79.965902,3.5,57,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsAt...","Fast Food, Greek, Breakfast & Brunch, Restaura...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."


In [74]:
Hotel_index = np.where(raw_business.categories.str.contains('Hotel').values)[0]
Hotel_business = raw_business.iloc[Hotel_index,]
Hotel_business.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
1,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5,1,0,0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,Black Cab & Limo,...,IL,,40.116420,-88.243383,4.5,28,1,{'WiFi': 'u'free''},"Airport Shuttles, Transportation, Taxis, Hotel...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
3,2mLlQ9n5Ln_fykOuX_b0-A,EZ6g9a17UdMjtR-wrZebmA,wSAGILK2CFbq5BVZ5D67-A,3,0,0,0,The service is attentive. The decor attractive...,2015-06-21 00:07:59,Adega,...,OH,44115,41.499615,-81.685922,4.0,201,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Mediterranean, Hotels, Beer, Wine & Spirits, B...","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
5,nQxoDWJ5kIwjYZhFILqVWQ,lJBI50awi0N8qvX4PKzH7Q,3n685UjMonF4fjBcILXkJg,4,0,0,0,Room was very clean. Staff was exceptionally ...,2017-08-16 11:21:42,Hampton Inn Pittsburgh University/Medical Center,...,PA,15213,40.436035,-79.963010,4.0,60,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Hotels & Travel, Event Planning & Services, Ho...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
15,n2BEHit2SwqsRo6jlTagUA,abdw-Nf5xxWdi5wmUO7UYg,2qLOnQtWSlTOtkfEFyWScg,2,2,0,0,"There are good things, and bad things about th...",2013-04-10 19:26:54,University of Illinois Willard Airport,...,IL,61874,40.036489,-88.263988,4.0,38,1,{'WiFi': 'u'free''},"Hotels & Travel, Airports","{'Monday': '4:0-23:0', 'Tuesday': '4:0-23:0', ..."
20,mrK-GF4zYYj8A3oIlvUVTQ,wurJLbqM_iM0R64dQ8r3Qw,z9lOfnUorcbWR_xS0DSeHw,4,2,1,1,I came for a quick bite and just ordered from ...,2014-08-24 20:25:08,Fugu,...,WI,53703,43.074392,-89.393664,3.5,180,1,"{'RestaurantsDelivery': 'True', 'RestaurantsAt...","Szechuan, Thai, Chinese, Town Car Service, Tra...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [75]:
Hotel_Madison = Hotel_business[Hotel_business.city == 'Madison']
Hotel_Madison = Hotel_Madison.drop(columns=['city', 'state'])
Hotel_Madison.reset_index(drop=True, inplace=True)
Hotel_Madison.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,address,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,mrK-GF4zYYj8A3oIlvUVTQ,wurJLbqM_iM0R64dQ8r3Qw,z9lOfnUorcbWR_xS0DSeHw,4,2,1,1,I came for a quick bite and just ordered from ...,2014-08-24 20:25:08,Fugu,411 W Gilman St,53703,43.074392,-89.393664,3.5,180,1,"{'RestaurantsDelivery': 'True', 'RestaurantsAt...","Szechuan, Thai, Chinese, Town Car Service, Tra...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1,3s8uY7ucUBrovfujAd2TrQ,3p_YUoTXOe2aML4GbDsBdQ,3kQarjSJYYo9nGAkFjbWYg,3,0,0,0,I stayed here twice in a short period of time ...,2014-06-11 17:55:35,Fairfield Inn & Suites by Marriott Madison East,2702 Crossroads Dr,53718,43.133389,-89.291328,3.5,22,1,"{'RestaurantsPriceRange2': '2', 'WiFi': 'u'fre...","Hotels & Travel, Event Planning & Services, Ho...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
2,H0OlZnbjlCcLxZnUsp6p5g,g64lOV39xSLRZO0aQQ6DeQ,DHcbeJ-5tk2RQ4SdcdJqkA,4,1,0,1,Our room on the second floor was very nice. Th...,2015-01-18 15:02:00,Hotel Ruby Marie,524 E Wilson St,53703,43.076340,-89.376209,4.0,62,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Bed & Breakfast, Hotels & Travel, Hotels, Even...","{'Monday': '0:0-23:59', 'Tuesday': '0:0-23:59'..."
3,2YA139B8VrTjgxbAF5mLFA,vEo7fEwrEWIA4d9swwDasg,z9lOfnUorcbWR_xS0DSeHw,4,0,0,0,Definitely will go back...best wonton soup I'v...,2016-07-03 19:19:40,Fugu,411 W Gilman St,53703,43.074392,-89.393664,3.5,180,1,"{'RestaurantsDelivery': 'True', 'RestaurantsAt...","Szechuan, Thai, Chinese, Town Car Service, Tra...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
4,BeaI_I2S6EJGctncfew89A,t4idL2iqySViutbKeZBXpQ,EEB9X2j6jUm2lF75LAo4Xw,1,0,0,0,Turns out to be just as shady as uber. They l...,2017-01-02 04:04:33,Lyft,,35715,43.073052,-89.401230,2.0,4,1,None,"Automotive, Hotels & Travel, Taxis, Transporta...","{'Monday': '9:0-9:0', 'Tuesday': '9:0-9:0', 'W..."


In [76]:
Hotel_Madison.shape

(4074, 20)

In [77]:
Hotel_Madison.columns

Index(['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny',
       'cool', 'text', 'date', 'name', 'address', 'postal_code', 'latitude',
       'longitude', 'stars_y', 'review_count', 'is_open', 'attributes',
       'categories', 'hours'],
      dtype='object')

In [78]:
Hotel_Madison = Hotel_Madison.drop(columns=['business_id', 'review_id', 'user_id', 'funny', 'cool', 'date', 'latitude', 'longitude', 'is_open', 'review_count', 'hours'])
Hotel_Madison.head()

,stars_x,useful,text,name,address,postal_code,stars_y,attributes,categories
0,4,2,I came for a quick bite and just ordered from ...,Fugu,411 W Gilman St,53703,3.5,"{'RestaurantsDelivery': 'True', 'RestaurantsAt...","Szechuan, Thai, Chinese, Town Car Service, Tra..."
1,3,0,I stayed here twice in a short period of time ...,Fairfield Inn & Suites by Marriott Madison East,2702 Crossroads Dr,53718,3.5,"{'RestaurantsPriceRange2': '2', 'WiFi': 'u'fre...","Hotels & Travel, Event Planning & Services, Ho..."
2,4,1,Our room on the second floor was very nice. Th...,Hotel Ruby Marie,524 E Wilson St,53703,4.0,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Bed & Breakfast, Hotels & Travel, Hotels, Even..."
3,4,0,Definitely will go back...best wonton soup I'v...,Fugu,411 W Gilman St,53703,3.5,"{'RestaurantsDelivery': 'True', 'RestaurantsAt...","Szechuan, Thai, Chinese, Town Car Service, Tra..."
4,1,0,Turns out to be just as shady as uber. They l...,Lyft,,35715,2.0,None,"Automotive, Hotels & Travel, Taxis, Transporta..."


In [79]:
Hotel_Madison.shape

(4074, 9)

In [80]:
text = list(Hotel_Madison.text)

In [81]:
text = [a.lower() for a in text]
text = ["".join([char for char in a if char not in string.punctuation]) for a in text]

In [82]:
words = [nltk.word_tokenize(a) for a in text]

In [83]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
filtered_words = [[word for word in a if word not in stop_words] for a in words]

In [84]:
from nltk.stem import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [85]:
lemmatized_words = [[lmtzr.lemmatize(x) for x in a] for a in filtered_words]

In [86]:
word_list = list(itertools.chain.from_iterable(lemmatized_words))
from collections import Counter
freq = pd.DataFrame.from_dict(Counter(word_list), orient='index')

In [87]:
freq.sort_values(by=0, ascending=False)

,0
room,4583
hotel,3557
stay,1802
time,1773
great,1740
...,...
throughput,1
swamped,1
fixwhen,1
excusedid,1


In [88]:
dictionary = list(freq[freq[0] > 10].index)

In [89]:
word_embedding = np.zeros([len(lemmatized_words), len(dictionary)])

In [90]:
for i,x in enumerate(lemmatized_words):
    for j,y in enumerate(dictionary):
        word_embedding[i,j] = sum([z == y for z in x])

In [91]:
word_embedding

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [92]:
df = pd.concat([Hotel_Madison, pd.DataFrame(word_embedding, columns=dictionary)], axis=1)

In [93]:
df

,stars_x,useful,text,name,address,postal_code,stars_y,attributes,categories,came,...,getz,llama,sheep,sampled,tsa,precheck,flown,ord,americinn,arbor
0,4,2,I came for a quick bite and just ordered from ...,Fugu,411 W Gilman St,53703,3.5,"{'RestaurantsDelivery': 'True', 'RestaurantsAt...","Szechuan, Thai, Chinese, Town Car Service, Tra...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0,I stayed here twice in a short period of time ...,Fairfield Inn & Suites by Marriott Madison East,2702 Crossroads Dr,53718,3.5,"{'RestaurantsPriceRange2': '2', 'WiFi': 'u'fre...","Hotels & Travel, Event Planning & Services, Ho...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1,Our room on the second floor was very nice. Th...,Hotel Ruby Marie,524 E Wilson St,53703,4.0,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Bed & Breakfast, Hotels & Travel, Hotels, Even...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,Definitely will go back...best wonton soup I'v...,Fugu,411 W Gilman St,53703,3.5,"{'RestaurantsDelivery': 'True', 'RestaurantsAt...","Szechuan, Thai, Chinese, Town Car Service, Tra...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,Turns out to be just as shady as uber. They l...,Lyft,,35715,2.0,None,"Automotive, Hotels & Travel, Taxis, Transporta...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069,1,1,I have never posted a negative review on any s...,Americas Best Value Inn Madison,3438 Highway 12-18,53704,1.5,"{'RestaurantsPriceRange2': '1', 'WiFi': 'u'fre...","Hotels & Travel, Hotels, Event Planning & Serv...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4070,1,0,I rent 200 plus days a year. Was offered an Im...,National Car Rental,4000 International Ln,53704,2.0,None,"Car Rental, Hotels & Travel",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4071,5,1,This was one of the coolest hotels I've stayed...,Hotel Indigo Madison Downtown,901 E Washington Ave,53703,5.0,"{'BusinessAcceptsBitcoin': 'False', 'Wheelchai...","Venues & Event Spaces, Hotels & Travel, Hotels...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4072,1,0,The worst National I've been to across the who...,National Car Rental,4000 International Ln,53704,2.0,None,"Car Rental, Hotels & Travel",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
df.to_csv('df.csv')